### Import Libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Bring data in from csv to dataframes

#### Bring in home prices data

In [2]:
home_prices_file = "Datasets/01-Zillow_Dataset.csv"
home_prices_df = pd.read_csv(home_prices_file)
home_prices_df.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


#### Bring in investment data

In [3]:
investment_file = "Datasets/Clean_Startup_data.csv"
investment_df = pd.read_csv(investment_file, encoding='unicode_escape')
investment_df.head()

,Unnamed: 0,name,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds
0,0,#waywire,Entertainment Politics Social Media News,News,"17,50,000",acquired,USA,NY,New York City,New York,1.0
1,1,&TV Communications,Games,Games,"40,00,000",operating,USA,CA,Los Angeles,Los Angeles,2.0
2,2,'Rock' Your Paper,Publishing Education,Publishing,"40,000",operating,EST,NaN,Tallinn,Tallinn,1.0
3,3,(In)Touch Network,Electronics Guides Coffee Restaurants Music i...,Electronics,"15,00,000",operating,GBR,NaN,London,London,1.0
4,4,-R- Ranch and Mine,Tourism Entertainment Games,Tourism,"60,000",operating,USA,TX,Dallas,Fort Worth,2.0


### Clean the data

In [4]:
# Create a filtered dataframe from specific columns
home_prices_cols = ["RegionName",
                    "State",
                    "Metro",
                    "SizeRank",
                    "2016-08"]
home_prices_transformed= home_prices_df[home_prices_cols].copy()

# Rename the column headers
home_prices_transformed = home_prices_transformed.rename(columns={
    "RegionName": "region",
    "State": "state_code",
    "Metro": "metro_area",
    "SizeRank": "size_rank",
    "2016-08": "house_value"
})
home_prices_transformed.index.name = 'id'
home_prices_transformed.head()

,region,state_code,metro_area,size_rank,house_value
id,,,,,
0,New York,NY,New York,1,586400
1,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,2,585100
2,Chicago,IL,Chicago,3,213000
3,Philadelphia,PA,Philadelphia,4,129100
4,Phoenix,AZ,Phoenix,5,195900


In [5]:
# Create a filtered dataframe from specific columns
investment_cols = ["name",
                    "category_list",
                    "market",
                    "funding_total_usd",
                    "status",
                    "country_code",
                    "state_code",
                    "region",
                    "city",
                    "funding_rounds"
                  ]
investment_transformed= investment_df[investment_cols].copy()

# Rename the column headers
investment_transformed = investment_transformed.rename(columns={
    "funding_total_usd": "funding_total"
})
investment_transformed.index.name = 'id'
investment_transformed.head()

,name,category_list,market,funding_total,status,country_code,state_code,region,city,funding_rounds
id,,,,,,,,,,
0,#waywire,Entertainment Politics Social Media News,News,"17,50,000",acquired,USA,NY,New York City,New York,1.0
1,&TV Communications,Games,Games,"40,00,000",operating,USA,CA,Los Angeles,Los Angeles,2.0
2,'Rock' Your Paper,Publishing Education,Publishing,"40,000",operating,EST,NaN,Tallinn,Tallinn,1.0
3,(In)Touch Network,Electronics Guides Coffee Restaurants Music i...,Electronics,"15,00,000",operating,GBR,NaN,London,London,1.0
4,-R- Ranch and Mine,Tourism Entertainment Games,Tourism,"60,000",operating,USA,TX,Dallas,Fort Worth,2.0


### Load the cleaned data into our database

#### Connect to database

In [6]:
rds_connection_string = "postgres:postgres@localhost:5432/etl_project_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [7]:
# Confirm tables
engine.table_names()

['investment_data', 'home_price_data']

#### Load dataframes into database

In [8]:
home_prices_transformed.to_sql(name='home_price_data', con=engine, if_exists='append', index=True)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "home_price_data_pkey"
DETAIL:  Key (id)=(0) already exists.

[SQL: INSERT INTO home_price_data (id, region, state_code, metro_area, size_rank, house_value) VALUES (%(id)s, %(region)s, %(state_code)s, %(metro_area)s, %(size_rank)s, %(house_value)s)]
[parameters: ({'id': 0, 'region': 'New York', 'state_code': 'NY', 'metro_area': 'New York', 'size_rank': 1, 'house_value': 586400}, {'id': 1, 'region': 'Los Angeles', 'state_code': 'CA', 'metro_area': 'Los Angeles-Long Beach-Anaheim', 'size_rank': 2, 'house_value': 585100}, {'id': 2, 'region': 'Chicago', 'state_code': 'IL', 'metro_area': 'Chicago', 'size_rank': 3, 'house_value': 213000}, {'id': 3, 'region': 'Philadelphia', 'state_code': 'PA', 'metro_area': 'Philadelphia', 'size_rank': 4, 'house_value': 129100}, {'id': 4, 'region': 'Phoenix', 'state_code': 'AZ', 'metro_area': 'Phoenix', 'size_rank': 5, 'house_value': 195900}, {'id': 5, 'region': 

In [9]:
investment_transformed.to_sql(name='investment_data', con=engine, if_exists='append', index=True)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "investment_data_pkey"
DETAIL:  Key (id)=(0) already exists.

[SQL: INSERT INTO investment_data (id, name, category_list, market, funding_total, status, country_code, state_code, region, city, funding_rounds) VALUES (%(id)s, %(name)s, %(category_list)s, %(market)s, %(funding_total)s, %(status)s, %(country_code)s, %(state_code)s, %(region)s, %(city)s, %(funding_rounds)s)]
[parameters: ({'id': 0, 'name': '#waywire', 'category_list': ' Entertainment Politics Social Media News ', 'market': ' News ', 'funding_total': ' 17,50,000 ', 'status': 'acquired', 'country_code': 'USA', 'state_code': 'NY', 'region': 'New York City', 'city': 'New York', 'funding_rounds': 1.0}, {'id': 1, 'name': '&TV Communications', 'category_list': ' Games ', 'market': ' Games ', 'funding_total': ' 40,00,000 ', 'status': 'operating', 'country_code': 'USA', 'state_code': 'CA', 'region': 'Los Angeles', 'city': 'Los Angeles',

#### Confirm the loads

In [10]:
pd.read_sql_query('select * from home_price_data', con=engine).head()

,id,region,state_code,metro_area,size_rank,house_value
0,0,New York,NY,New York,1,586400
1,1,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,2,585100
2,2,Chicago,IL,Chicago,3,213000
3,3,Philadelphia,PA,Philadelphia,4,129100
4,4,Phoenix,AZ,Phoenix,5,195900


In [ ]:
pd.read_sql_query('select * from investment_data', con=engine).head()